In [45]:
import psycopg2
import pandas as pd
import altair as alt

In [1]:
import configparser

config = configparser.ConfigParser()
config.read('TEST.ini', encoding="utf-8")

config = {i: config['postgresql'][i].replace('"', '') for i in config['postgresql']}

In [49]:
connect = psycopg2.connect(**config)

cursor = connect.cursor()

cursor.execute("""
SELECT date,
       orders_count,
       ROUND(AVG(orders_count) OVER (
                                ORDER BY date ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING), 2) AS moving_avg
FROM (
    SELECT DATE(creation_time) AS date,
           COUNT(order_id) AS orders_count
    FROM orders
    WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
    GROUP BY date
) t;
""")

desc = [desc[0] for desc in cursor.description]

data_in_query = [{desc[j]: i[j] for j in range(len(desc))} for i in cursor.fetchall()]

print(data_in_query)

[{'date': datetime.date(2022, 8, 24), 'orders_count': 138, 'moving_avg': None}, {'date': datetime.date(2022, 8, 25), 'orders_count': 1059, 'moving_avg': Decimal('138.00')}, {'date': datetime.date(2022, 8, 26), 'orders_count': 1447, 'moving_avg': Decimal('598.50')}, {'date': datetime.date(2022, 8, 27), 'orders_count': 2141, 'moving_avg': Decimal('881.33')}, {'date': datetime.date(2022, 8, 28), 'orders_count': 2998, 'moving_avg': Decimal('1549.00')}, {'date': datetime.date(2022, 8, 29), 'orders_count': 3267, 'moving_avg': Decimal('2195.33')}, {'date': datetime.date(2022, 8, 30), 'orders_count': 3371, 'moving_avg': Decimal('2802.00')}, {'date': datetime.date(2022, 8, 31), 'orders_count': 3410, 'moving_avg': Decimal('3212.00')}, {'date': datetime.date(2022, 9, 1), 'orders_count': 3688, 'moving_avg': Decimal('3349.33')}, {'date': datetime.date(2022, 9, 2), 'orders_count': 5001, 'moving_avg': Decimal('3489.67')}, {'date': datetime.date(2022, 9, 3), 'orders_count': 5709, 'moving_avg': Decimal

In [50]:
def data():
    df = pd.DataFrame(data_in_query)
    return df

df = data()

df['date'] = df['date'].astype('string')
df['moving_avg'] = pd.to_numeric(df['moving_avg'])

print(df.dtypes)
print(df)

date            string[python]
orders_count             int64
moving_avg             float64
dtype: object
          date  orders_count  moving_avg
0   2022-08-24           138         NaN
1   2022-08-25          1059      138.00
2   2022-08-26          1447      598.50
3   2022-08-27          2141      881.33
4   2022-08-28          2998     1549.00
5   2022-08-29          3267     2195.33
6   2022-08-30          3371     2802.00
7   2022-08-31          3410     3212.00
8   2022-09-01          3688     3349.33
9   2022-09-02          5001     3489.67
10  2022-09-03          5709     4033.00
11  2022-09-04          6010     4799.33
12  2022-09-05          4675     5573.33
13  2022-09-06          3451     5464.67
14  2022-09-07          4777     4712.00
15  2022-09-08          5474     4301.00


    Посмотреть схему цветов
https://vega.github.io/vega/docs/schemes/

In [51]:
max_orders_count = df['orders_count'].max()

alt.Chart(df).mark_bar().encode(
    x='orders_count',
    y='date:O',
    color=alt.condition(
        alt.datum.orders_count == max_orders_count,
        alt.value('#bcdb22'),  # Цвет для максимального значения
        alt.value('#7f7f7f')  # Цвет для остальных значений
    )
).properties(width=800)

alt.Chart(...)

    Видео-гайд по altair в Jupyter Notebook:
https://www.youtube.com/watch?v=YCO5Zl2iY8k 

    Как создать таблицу по данным графика Altair:
https://stackoverflow.com/questions/67997825/python-altair-generate-a-table-on-selection

    Кодирование типов данных в указании name_column:TYPE 
https://altair-viz.github.io/altair-viz-v4/user_guide/encoding.html

    Как сделать название для графика Altair:
https://www.appsloveworld.com/coding/python3x/6/how-to-add-a-subtitle-to-an-altair-generated-chart 